# Contact tracing simulation

**Import libs**

In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
if '..' not in sys.path:
    sys.path.append('..')
sys.path.insert(0, '../../sib/')
sys.path.insert(0, '../../../sib/')

In [2]:
import numpy as np
import random as rd
import pandas as pd
import pickle
import multiprocessing
import argparse
from lib.measures import *
from lib.experiment import Experiment, options_to_str, process_command_line
from lib.calibrationSettings import calibration_lockdown_dates, calibration_mob_paths, calibration_states, calibration_lockdown_beta_multipliers
from lib.calibrationFunctions import get_calibrated_params

from ranker import winbp_prob0_rank

import sib

import matplotlib.pyplot as plt
from lib.plot import Plotter
%matplotlib inline

# converting days to hours
TO_HOURS = 24.0


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


## 1. Define the experiment parameters

Some model parameters are in `calibrationSettings.py`, as the testing parameters (also for contact tracing), the lockdown dates, the path to the .pk file and the beta multipliers for a lockdown.

In `calibration_testing_params`, `'test_queue_policy'` can be set to either `'fifo'` (first in first out) or `'exposure-risk'` (priority based), `'smart_tracing_actions'` can be any of `'isolate'` and `'test'`, while `'smart_tracing_policy_isolate'` and `smart_tracing_policy_test` are both set to `'sib'` in order to use the sib contact tracing. These values are set in the list `settings`:

In [3]:
name = 'rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000'

country = 'GER'
area = 'TU'
start_date = '2020-07-01'
#end_date = '2020-07-31'
end_date = '2020-07-08'

random_repeats = 1 # recommended to set to at least 48  
full_scale = False

# Load mob settings        
mob_settings_file = calibration_mob_paths[country][area][1 if full_scale else 0]
with open(mob_settings_file, 'rb') as fp:
    mob_settings = pickle.load(fp)

N = int(mob_settings['num_people_unscaled'] / mob_settings['downsample'])


# list of contact tracing experiments parameters
# (test_queue_policy, smart_tracing_actions, tests_per_batch, test_delay, smart_tracing_contacts, test_isolation_policy)
settings = [
    #('fifo', [], 0, 48.0, 0, None),
    #('fifo', ['isolate', 'test'], 5000, 48.0, 5000, 'sib'),
    #('fifo', ['isolate', 'test'], 5000, 48.0, 5000, 'basic')
    ('fifo', [], 0, 48.0, 0, None),
    ('fifo', ['isolate', 'test'], 500, 48.0, 500, 'sib'),
#     ('fifo', ['isolate', 'test'], 1000, 48.0, 1000, 'sib'),
#     ('fifo', ['isolate', 'test'], 1500, 48.0, 1500, 'sib'),
]

#names = ['no_testing', 'sib_tracing', 'article_tracing']
names = ['no_testing', 'sib_tracing500', 'sib_tracing1000', 'sib_tracing1500']

# seed
c = 0
np.random.seed(c)
rd.seed(c)

The parameter `full_scale` is used to select between the data with downsapling and the data without: for example in case of `area = 'TU'` (Tübingen), in `calibrationSettings.py` there is:

    # mobs settings path;
    calibration_mob_paths = {
        'GER': {
            'TU': ['lib/mobility/Tubingen_settings_10.pk', 'lib/mobility/Tubingen_settings_1.pk'],
            #'TU': ['lib/mobility/Tubingen_settings_10_beacon.pk', 'lib/mobility/Tubingen_settings_1_beacon.pk'],
            'KL': ['lib/mobility/Kaiserslautern_settings_10.pk', 'lib/mobility/Kaiserslautern_settings_1.pk'],
            'RH': ['lib/mobility/Ruedesheim_settings_10.pk', 'lib/mobility/Ruedesheim_settings_1.pk'],
            'TR': ['lib/mobility/Tirschenreuth_settings_5.pk', 'lib/mobility/Tirschenreuth_settings_1.pk'],
        }
        ### OTHER LINES ###
    }

and so `full_scale = False` selects `Tubingen_settings_10.pk`, while `full_scale = True` selects `Tubingen_settings_1.pk`.

The model parameters should be calibrated using Bayesian optimization. 
Run `python calibrate.py --help` to see how to use the calibration script. Also, refer to `calibrationSettings.py` for detailed settings and automatic loading of results.

#### Add ranker

In [4]:
mu = 1/12
prob_seed = 1/N
prob_sus = 0.5
pautoinf = 1e-10
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1e-6
fp_rate = 0.0
fn_rate = 0.0

ranker = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Uniform(1.0), 
                                prob_r = sib.Exponential(0.1), 
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=0
)

#### Model parameters

In [5]:
arbitrary_model_params = {
    'beta_site' :  0.55,      # risk of exposure at sites
    'beta_household' : 0.55,  # risk of exposure at home
    'p_stay_home' : 1.00,     # probability of not attending a given visit to a site due to social distancing
}

seed_counts = {
        'expo': 3,
#         'isym_posi': int(np.round(isym).item()),
#         'iasy': int(np.round(iasy).item()),
    }

#### Define Experiment object

In [6]:
experiment_info = f'{name}-{country}-{area}'
experiment = Experiment(
    experiment_info=experiment_info,
    start_date=start_date,
    end_date=end_date,
    random_repeats=random_repeats,
    full_scale=full_scale,
    verbose=True,
    ranker=ranker
)

#### Loop over all simulations

The measures to take place during simulation are also defined here, and the simulation is added to the experiment.

For example to add a lockdown starting from day 14:

    lockdown_at_day = 14

    sim_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
    max_time = TO_HOURS * sim_days # in hours

    example_measures = [

        # education, social sites, and offices partially closed/reduced infection after 1 week
        BetaMultiplierMeasureByType(
            t_window=Interval(lockdown_at_day * TO_HOURS, max_time), 
            beta_multiplier={ 
                'education': 0.5, 
                'social': 0.5, 
                'bus_stop': 1.0, 
                'office': 0.5, 
                'supermarket': 1.0
            }),

        # less activities of all due to contact constraints after 14 days
        SocialDistancingForAllMeasure(
         t_window=Interval(lockdown_at_day * TO_HOURS, max_time), 
            p_stay_home=arbitrary_model_params['p_stay_home'])
    ]

and the elements of `example_measures` must be added to `m`.

In [7]:
count = 0

for (test_queue_policy,
     smart_tracing_actions,
     tests_per_batch,
     test_delay,
     smart_tracing_contacts,
     test_isolation_policy) in settings:
    
    # set testing params via update function of standard testing parameters
    def test_update(d):
        d['test_queue_policy'] = test_queue_policy
        d['smart_tracing_actions'] = smart_tracing_actions
        d['test_reporting_lag'] = test_delay
        d['tests_per_batch'] = tests_per_batch

        # isolation
        d['smart_tracing_policy_isolate'] = test_isolation_policy
        d['smart_tracing_isolated_contacts'] = smart_tracing_contacts
        d['smart_tracing_isolation_duration'] = 14 * TO_HOURS

        # testing
        d['smart_tracing_policy_test'] = test_isolation_policy
        d['smart_tracing_tested_contacts'] = smart_tracing_contacts

        return d
    
    # measures
    max_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
    
    m = [
            SocialDistancingForSmartTracing(
                t_window=Interval(0.0, TO_HOURS * max_days), 
                p_stay_home=1.0, 
                smart_tracing_isolation_duration=TO_HOURS * 14.0),
            SocialDistancingForSmartTracingHousehold(
                t_window=Interval(0.0, TO_HOURS * max_days),
                p_isolate=1.0,
                smart_tracing_isolation_duration=TO_HOURS * 14.0),
            SocialDistancingSymptomaticAfterSmartTracing(
                t_window=Interval(0.0, TO_HOURS * max_days),
                p_stay_home=1.0,
                smart_tracing_isolation_duration=TO_HOURS * 14.0),
            SocialDistancingSymptomaticAfterSmartTracingHousehold(
                t_window=Interval(0.0, TO_HOURS * max_days),
                p_isolate=1.0,
                smart_tracing_isolation_duration=TO_HOURS * 14.0),
            ]
    
    simulation_info = options_to_str(
        descr=names[count],
    )
    
    count += 1

    experiment.add(
        simulation_info=simulation_info,
        country=country,
        area=area,
        measure_list=m,
        lockdown_measures_active = False,
        set_calibrated_params_to=arbitrary_model_params,
        full_scale=full_scale,
        test_update=test_update,
        seed_summary_path=None,
        set_initial_seeds_to=seed_counts)

    print(f'{experiment_info} configuration done.')

[Added Sim] rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU/rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU-descr=no_testing
rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU configuration done.
[Added Sim] rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU/rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU-descr=sib_tracing500
rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU configuration done.


In `experiment.add()` there is also the possibility to specify other parameters of the simulation, such as `expected_daily_base_expo_per100k`, which is by default set to 0, while setting a nonzero value has the following effect:

    if expected_daily_base_expo_per100k > 0.0:

        # Scale expectation to simulation size
        num_people = len(mob_settings['home_loc'])
        lambda_base_expo_population = expected_daily_base_expo_per100k * (num_people / 100000)

        # Convert to individual base rate by dividing by population size; priority queue handles superposition
        lambda_base_expo_indiv = lambda_base_expo_population / num_people

        # Poisson process with rate lambda: interarrival times are Exponential r.v. with mean = 1 / lambda
        # Hence set rate of Expo r.v.s to 1 / (1 / lambda) = lambda
        distributions.lambda_0 = lambda_base_expo_indiv

Another parameter that can be specified is `beacon_config`, which can be `None` (digital tracing is P2P) or `beacon_config['mode']` can be `'all'` (all sites have a beacon), `'random'` (some sites have a beacon) or `'visit_freq'` (sites with higher priority have a beacon, where the priority is based on integrated visit time scaled with site specific beta). By default `beacon_config` is `None`.

## 2. Run the simulations

Run the experiment using the parameters defined above and simulate in the future, with additional measures of varying duration.

**WARNING: this cell might take a long time to run depending of the parameters defined above**

In [8]:
print('Warning: debug mode in parallel.py')
experiment.run_all() 

Starting launch_parallel_simulations
Starting pp_launch
cpu_count: 1
repeat_ids: [0]
Inside pp_launch
Simulating mobility
Min duration with delta: 0.36541861300374556
Max duration with delta: 20.587981448563976
Mobility simulation ended
Using delta: 0.3654120904376099
Launching epidemic
Initializing inference algorithm
Main loop starting, population of 9054

Testing event at time 24.0
Tested 0 individuals

Testing event at time 48.0
Tested 0 individuals

Testing event at time 72.0
Tested 0 individuals

Testing event at time 96.0
Tested 0 individuals

Testing event at time 120.0
Tested 0 individuals

Testing event at time 144.0
Tested 0 individuals
End main loop
Total number of infections: 84
Infections from contacts 57
Infections from indirect contacts 3
Infections from pure indirect contacts 0
[Finished Sim] rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU/rescale-seedcomp-1repeats-50days-10downsampling-0.55beta-0000-GER-TU-descr=no_testing
Starting launch_parallel

## 3. Plot the results

Refer to `sim-plot.ipynb` for plotting. The results of the above experiment are stored inside the `sim/summaries/` folder of the repository.

# TO CHECK

* why is `calibration_states` imported?
* ranker needs `prob_seed = 1/N`; atm N is `'num_people_unscaled'`, should it be the scaled number of people?